In [6]:
with open("human_chat.txt",'rb') as file:
    text = file.read().lower()
print(text)

b'human 1: hi!\nhuman 2: what is your favorite holiday?\nhuman 1: one where i get to meet lots of different people.\nhuman 2: what was the most number of people you have ever met during a holiday?\nhuman 1: hard to keep a count. maybe 25.\nhuman 2: which holiday was that?\nhuman 1: i think it was australia\nhuman 2: do you still talk to the people you met?\nhuman 1: not really. the interactions are usually short-lived but it\'s fascinating to learn where people are coming from and what matters to them\nhuman 2: yea, me too. i feel like god often puts strangers in front of you, and gives you an opportunity to connect with them in that moment in deeply meaningful ways. do you ever feel like you know things about strangers without them telling you?\nhuman 1: what do you mean?\nhuman 2: i think it\'s like a 6th sense, often seen as "cold readings" to people, but can be remarkably accurate. i once sat next to a man in a coffee and i felt a pain in my back. i asked the stranger if he had a p

In [7]:
import nltk
nltk.download("all")
from nltk.tokenize import sent_tokenize
text=str(text)
sentences=sent_tokenize(text)
print(sentences)

["b'human 1: hi!\\nhuman 2: what is your favorite holiday?\\nhuman 1: one where i get to meet lots of different people.\\nhuman 2: what was the most number of people you have ever met during a holiday?\\nhuman 1: hard to keep a count.", 'maybe 25.\\nhuman 2: which holiday was that?\\nhuman 1: i think it was australia\\nhuman 2: do you still talk to the people you met?\\nhuman 1: not really.', "the interactions are usually short-lived but it\\'s fascinating to learn where people are coming from and what matters to them\\nhuman 2: yea, me too.", 'i feel like god often puts strangers in front of you, and gives you an opportunity to connect with them in that moment in deeply meaningful ways.', 'do you ever feel like you know things about strangers without them telling you?\\nhuman 1: what do you mean?\\nhuman 2: i think it\\\'s like a 6th sense, often seen as "cold readings" to people, but can be remarkably accurate.', 'i once sat next to a man in a coffee and i felt a pain in my back.', '

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /home/ri_rishna_urandugp/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/ri_rishna_urandugp/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/ri_rishna_urandugp/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /home/ri_rishna_urandugp/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/ri_rishna_urandugp/nltk_data...
[nltk_data]    |   Package aver

In [8]:
import re
cleaned_sentence=[]
for sentence in sentences:
    sentence = sentence.replace("\\n", " ")
    sentence = sentence.replace(":", " ")
    sentence = re.sub(r"^b[\'\"]", '', sentence)
    sentence = re.sub(r'\bhuman\s*1\b', '', sentence, flags=re.IGNORECASE)
    sentence = re.sub(r'\bhuman\s*2\b', '', sentence, flags=re.IGNORECASE)
    sentence = sentence.replace("/", "")
    sentence = sentence.strip().strip('"').strip("'").replace("\\", "")
    if sentence:  # only append if it's not empty
        cleaned_sentence.append(sentence)

cleaned_sentence

['hi!   what is your favorite holiday?   one where i get to meet lots of different people.   what was the most number of people you have ever met during a holiday?   hard to keep a count.',
 'maybe 25.   which holiday was that?   i think it was australia   do you still talk to the people you met?   not really.',
 "the interactions are usually short-lived but it's fascinating to learn where people are coming from and what matters to them   yea, me too.",
 'i feel like god often puts strangers in front of you, and gives you an opportunity to connect with them in that moment in deeply meaningful ways.',
 'do you ever feel like you know things about strangers without them telling you?   what do you mean?   i think it\'s like a 6th sense, often seen as "cold readings" to people, but can be remarkably accurate.',
 'i once sat next to a man in a coffee and i felt a pain in my back.',
 'i asked the stranger if he had a pain.',
 'it turns out that he did in the exact spot, and said he pulled a 

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils import to_categorical

tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_sentence)


sequences = []
for line in cleaned_sentence:
    tokenized = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(tokenized)):
        sequences.append(tokenized[:i+1])

max_len_seq=max(len(seq) for seq in sequences)
sequences= pad_sequences(sequences,maxlen=max_len_seq)

sentences=np.array(sequences)
X=sequences[:, :-1]
y = to_categorical(sequences[:, -1], num_classes=len(tokenizer.word_index)+1)



In [12]:

from keras.models import Sequential
from keras.layers import Dense ,GRU,Embedding

model=Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1,output_dim=50,input_length=max_len_seq-1))
model.add(GRU(200))
model.add(Dense(activation="softmax",units=len(tokenizer.word_index)+1))

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

history = model.fit(X, y, epochs=100, verbose=1)
# Accuracy and loss per epoch
train_acc = history.history['accuracy']
train_loss = history.history['loss']


Epoch 1/100
570/570 [==============================] - 13s 22ms/step - loss: 6.4995 - accuracy: 0.0480
Epoch 2/100
570/570 [==============================] - 7s 12ms/step - loss: 5.8210 - accuracy: 0.0870
Epoch 3/100
570/570 [==============================] - 6s 11ms/step - loss: 5.3930 - accuracy: 0.1160
Epoch 4/100
570/570 [==============================] - 6s 11ms/step - loss: 4.9831 - accuracy: 0.1424
Epoch 5/100
570/570 [==============================] - 6s 10ms/step - loss: 4.5590 - accuracy: 0.1634
Epoch 6/100
570/570 [==============================] - 12s 22ms/step - loss: 4.1142 - accuracy: 0.1969
Epoch 7/100
570/570 [==============================] - 6s 11ms/step - loss: 3.6657 - accuracy: 0.2508
Epoch 8/100
570/570 [==============================] - 6s 10ms/step - loss: 3.2450 - accuracy: 0.3142
Epoch 9/100
570/570 [==============================] - -0s -256us/step - loss: 2.8611 - accuracy: 0.3843
Epoch 10/100
570/570 [==============================] - 6s 11ms/step - loss: 

In [16]:
print(max(train_acc))

0.9120613932609558


In [17]:
model.save("Predictive_typing_GRU.keras")  #accuracy=0.9110

In [18]:
def predict_next_word(model,seed_text,max_sequence_len):
    tokens=tokenizer.texts_to_sequences([seed_text])[0]
    tokens=pad_sequences([tokens],maxlen=max_sequence_len-1)
    predicted_probs=model.predict(tokens,verbose=0)
    predicted_index=np.argmax(predicted_probs,axis=-1)[0]
    predicted_word=tokenizer.index_word.get(predicted_index,"")
    return predicted_word

In [24]:
import pickle
with open("max_len_seq.pkl","wb") as file:
    pickle.dump(max_len_seq,file)
with open("tokenizer.pkl", "wb") as file:
    pickle.dump(tokenizer,file)
